# Implement CNN network with TensorFlow
## Brief
Load a CNN implemented in TensorFlow trained before and test it's performance on MNIST dataset.

## Notice
* Saving checkpoints of this model demands sufficient disk memory.

### Import necessary libraries

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

### Load MNIST dataset

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
MNIST=input_data.read_data_sets(r"..\DataSet\mnist",one_hot=True)
print("Number of training samples: {}\nNumber of test samples: {}".format(MNIST.train.num_examples,MNIST.test.num_examples))

Extracting ..\DataSet\mnist\train-images-idx3-ubyte.gz
Extracting ..\DataSet\mnist\train-labels-idx1-ubyte.gz
Extracting ..\DataSet\mnist\t10k-images-idx3-ubyte.gz
Extracting ..\DataSet\mnist\t10k-labels-idx1-ubyte.gz
Number of training samples: 55000
Number of test samples: 10000


### Create Graph

In [3]:
with tf.name_scope("CNN"):
    with tf.name_scope("Input"):
        X=tf.placeholder(dtype=tf.float32,shape=[None,784],name="X")
        Y=tf.placeholder(dtype=tf.float32,shape=[None,10],name="Y")
        NetIn=tf.reshape(tensor=X,shape=[-1,28,28,1],name="NetIn")
    with tf.name_scope("Conv1"):
        W1=tf.Variable(tf.truncated_normal([5,5,1,32]),name="Weight")
        b1=tf.Variable(tf.truncated_normal([32]),name="bias")
        L1Out=tf.nn.conv2d(input=NetIn,filter=W1,strides=[1,1,1,1],padding="SAME",name="conv")
        L1Out=tf.nn.relu(L1Out+b1,name="ReLu")
        L1Out=tf.nn.max_pool(value=L1Out,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name="L1Out")
        #Need to check out what the param#0 and param#3 means in ksize. 
    with tf.name_scope("Conv2"):
        W2=tf.Variable(tf.truncated_normal([5,5,32,64]),name="Weight")
        b2=tf.Variable(tf.truncated_normal([64]),name="bias")
        L2Out=tf.nn.conv2d(input=L1Out,filter=W2,strides=[1,1,1,1],padding="SAME",name="conv")
        L2Out=tf.nn.relu(L2Out+b2,name="ReLu")
        L2Out=tf.nn.max_pool(value=L2Out,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME",name="L2Out")
        #Need to check out what the param#0 and param#3 means in ksize.
    with tf.name_scope("FullConnect"):
        flatten=tf.reshape(tensor=L2Out,shape=[-1,7*7*64],name="flatten")
        W3=tf.Variable(tf.truncated_normal(dtype=tf.float32,shape=[7*7*64,1024]),name="Weight")
        b3=tf.Variable(tf.truncated_normal(dtype=tf.float32,shape=[1024]),name="bias")
        L3Out=tf.nn.relu(tf.matmul(flatten,W3)+b3,name="L3Out")
    with tf.name_scope("Output"):
        W4=tf.Variable(tf.truncated_normal(dtype=tf.float32,shape=[1024,10]),name="Weight")
        b4=tf.Variable(tf.truncated_normal(dtype=tf.float32,shape=[10]))
        L4Out=tf.matmul(L3Out,W4)+b4
    with tf.name_scope("Loss"):
        entropy=tf.nn.softmax_cross_entropy_with_logits(logits=L4Out,labels=Y,name="crossEntropy")
        loss=tf.reduce_mean(input_tensor=entropy,name="loss")
    with tf.name_scope("Train") as scope:
        optimizer=tf.train.AdamOptimizer(learning_rate=0.01,name="optimizer").minimize(loss)
        summary=tf.summary.scalar(name="lossSummary",tensor=loss)
        summary_op=tf.summary.merge(inputs=tf.get_collection(key=tf.GraphKeys.SUMMARIES,scope=scope))

### Reload Parameters and test performance

In [4]:
batchSize=20
with tf.Session() as sess:
    saver=tf.train.Saver()
    saver.restore(sess=sess,save_path=r".\model_checkpoints\MNIST_CNN-"+str(1000))
    acc=0
    for batch_i in range(int(MNIST.test.num_examples/batchSize)):
        x_batch,y_batch=MNIST.test.next_batch(batch_size=batchSize)
        pred=sess.run([L4Out,Y],feed_dict={X:x_batch})#Why should I feed Y?
        acc+=sess.run(tf.reduce_sum(tf.cast(x=tf.equal(tf.argmax(input=pred,axis=1),tf.argmax(input=y_batch,axis=1)),dtype=tf.float32)))
        print(acc/MNIST.test.num_examples)

InvalidArgumentError: You must feed a value for placeholder tensor 'CNN/Input/Y' with dtype float
	 [[Node: CNN/Input/Y = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'CNN/Input/Y', defined at:
  File "e:\python35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "e:\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "e:\python35\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "e:\python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "e:\python35\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "e:\python35\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "e:\python35\lib\site-packages\tornado\ioloop.py", line 887, in start
    handler_func(fd_obj,